In [2]:
import pandas as pd
import numpy as np
import video.df as df
from video import vid
from video import df
from video import frame
from video import preprocess as pr

import video.reader as r
import cv2 as cv
from video.reader import VideoReader


from typing import List, Union
from numpy.typing import ArrayLike

import matplotlib.pyplot as plt
from importlib import reload

2023-04-17 14:54:29.208365: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### check the potential hazard with zero-crossing algorithm

In [3]:
# get the lightness difference
v = pr.get_lightness_difference('videos/Banned Pokemon Seizure Scene.mp4')

In [6]:
v[:10]

array([ 0.0032,  0.0252,  0.024 ,  0.0388,  0.0064,  0.01  , -0.0044,
       -0.0076, -0.0064,  0.0044])

In [19]:
v1 = pr.get_lightness_difference1('videos/Banned Pokemon Seizure Scene.mp4')

In [ ]:
v1[:10]

array([-0.0032, -0.0252, -0.024 , -0.0388, -0.0064, -0.01  ,  0.0044,
        0.0076,  0.0064, -0.0044])

In [35]:
len(v), len(v1) # v1 is the same as v but opposite signs

(784, 784)

In [ ]:
np.sign(v)[:10]

array([ 1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.])

In [37]:
len(np.diff(np.sign(v)))

783

In [40]:
# if the next sign is the same the difference is going to be 0, els 2 or - 2
np.diff(np.sign(v))[:10]

array([ 0.,  0.,  0.,  0.,  0., -2.,  0.,  0.,  2.,  0.])

In [47]:
# get just indexes where the diff is 2 or -2
np.where(np.diff(np.sign(v)))[0][:10] + 1

array([ 6,  9, 12, 13, 24, 26, 30, 31, 36, 38])

In [45]:
v[:10]

array([ 0.0032,  0.0252,  0.024 ,  0.0388,  0.0064,  0.01  , -0.0044,
       -0.0076, -0.0064,  0.0044])

In [48]:
zero_crossings = np.where(np.diff(np.sign(v)))[0] + 1

In [49]:
zero_crossings[:10]

array([ 6,  9, 12, 13, 24, 26, 30, 31, 36, 38])

In [50]:
zero_crossings1 = np.where(np.diff(np.sign(v1)))[0] + 1

In [55]:
zero_crossings.shape[0]

422

In [53]:
# just double confirm that no matter the sign we get same result and 
# I can use simplier differencing
(zero_crossings1 == zero_crossings).sum()

422

In [56]:
(zero_crossings1 == zero_crossings).sum() == zero_crossings.shape[0]

True

Now we need to check if the lightness difference crosses 0 more than 3 times per second.

In [57]:
# we have 30 frames per second video
# if there is more than 3 consecutive zero crossings in 30 frames window
# it is a potential hazard
fps = 30

In [59]:
zero_crossings

array([  6,   9,  12,  13,  24,  26,  30,  31,  36,  38,  39,  40,  41,
        43,  45,  46,  48,  50,  52,  53,  54,  56,  57,  58,  59,  61,
        64,  66,  68,  69,  70,  71,  72,  74,  75,  76,  79,  80,  82,
        83,  86,  87,  88,  89,  92,  94,  95,  96,  97,  98,  99, 101,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116,
       118, 119, 120, 127, 128, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 143, 145, 147, 148, 151, 152, 153, 156, 157, 158,
       159, 160, 162, 163, 164, 167, 168, 169, 170, 171, 172, 173, 175,
       178, 180, 184, 185, 186, 187, 188, 189, 190, 191, 194, 195, 196,
       197, 198, 200, 201, 203, 206, 207, 208, 209, 210, 211, 212, 215,
       216, 217, 218, 219, 220, 221, 222, 223, 225, 226, 231, 232, 234,
       236, 240, 241, 243, 244, 248, 249, 254, 256, 257, 258, 259, 261,
       262, 267, 268, 270, 271, 272, 275, 276, 281, 283, 284, 285, 286,
       287, 295, 296, 299, 300, 301, 302, 305, 306, 308, 312, 31

In [ ]:
for i in range